In [6]:
import sys
sys.path.insert(0, '/users/fs2/hmehri/pythonproject/Thesis/synthetic')

from lib.field_info import FieldInfo_type2
from lib.tensor_encoder import TensorEncoder
from lib.prepare_data import preprocess_data_czech
from lib.modules import Encoder_Decoder_lstm, Encoder_Decoder_lstm_Inference
from sklearn.model_selection import train_test_split
from trainlstm import Train
import pandas as pd
import tensorflow as tf
import numpy as np
import unittest
import calendar
import datetime
pd.set_option('display.max_rows', 80)

In [8]:
import sys
sys.path.insert(0, '/users/fs2/hmehri/pythonproject/Thesis/synthetic')

from lib.prepare_data import preprocess_data_czech
from lib.field_info import FieldInfo, FIELD_INFO_TCODE, FieldInfo_type2
from lib.tensor_encoder import TensorEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from lib.modules import Encoder_Decoder_lstm
from trainlstm import Train
import json
import random
import os



def load_config(config_path):
    with open(config_path, 'r') as file:
        config = json.load(file)
    return config

def log_parameters(filename, parameters):
    log_entry = {'filename': filename, 'parameters': parameters}
    with open('parameter_log.json', 'a') as file:
        json.dump(log_entry, file)
        file.write('\n')  # New line for each entry


def make_batches(ds, buffer_size, batch_size):
    return ds.cache().shuffle(buffer_size).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

def create_tensor_dataset(encoder,bs, split=True):
    """bs is Batch Size
       if split=True, the input data is split into train and validation, otherwise the whole data is used for training """
    n_seqs, _, _ = encoder.inp_tensor.shape

    x_tr, x_cv, inds_tr, inds_cv, targ_tr, targ_cv = train_test_split(encoder.inp_tensor, np.arange(n_seqs), encoder.tar_tensor, test_size=0.2)

    # Create TensorFlow dataset
    ds_all = tf.data.Dataset.from_tensor_slices((encoder.inp_tensor.astype(np.float32), encoder.tar_tensor.astype(np.float32)))
    ds_tr = tf.data.Dataset.from_tensor_slices((x_tr.astype(np.float32), targ_tr.astype(np.float32)))
    ds_cv = tf.data.Dataset.from_tensor_slices((x_cv.astype(np.float32), targ_cv.astype(np.float32)))

    BUFFER_SIZE = ds_all.cardinality().numpy()

    all_batches =   make_batches(ds_all, BUFFER_SIZE, bs)
    train_batches = make_batches(ds_tr, BUFFER_SIZE, bs)
    val_batches =  make_batches(ds_cv, BUFFER_SIZE, bs)

    if split:
        return train_batches, val_batches
    else:
        return all_batches



with tf.device('/gpu:1'):
    raw_data = pd.read_csv('../DATA/tr_by_acct_w_age.csv')
    data, LOG_AMOUNT_SCALE, TD_SCALE,ATTR_SCALE, START_DATE, _ = preprocess_data_czech(raw_data)
    data2 = data[['account_id','age','age_sc', 'tcode', 'tcode_num', 'datetime', 'month', 'dow', 'day','td', 'dtme', 'log_amount','log_amount_sc','td_sc',
                                'type','operation', 'k_symbol', 'type_num', 'operation_num', 'k_symbol_num']]
    #data2 =  data[['account_id', 'tcode_num', 'age_sc', 'tcode', 'age']]
    df= data2.copy()

    confighyper = load_config('config_hyper.json')
    max_seq_len = confighyper['max_seq_len']
    min_seq_len = confighyper['min_seq_len']
    batch_size = confighyper['batch_size']
    epochs = confighyper['epochs'] 
    early_stop = confighyper['early_stop'] 
    len_generated_seq = confighyper['len_generated_seq'] 
    num_generated_seq = confighyper['num_generated_seq'] 
    synth_data_filename = confighyper["synth_data_filename"]
    strategy = confighyper['strategy']

    info = FieldInfo(strategy)
    #info = FIELD_INFO_TCODE()
    #info = FieldInfo_type2()

    
    encoder = TensorEncoder(df, info, max_seq_len, min_seq_len)
    encoder.encode_with_overlap(slide_step=80)

    n_seqs, seq_len, n_feat_inp = encoder.inp_tensor.shape
    raw_features = encoder.tar_tensor.shape[-1]    #7

    train_batches, val_batches = create_tensor_dataset(encoder, batch_size, split=True)



    config = {}
    config["ORDER"] = info.DATA_KEY_ORDER
    config["FIELD_STARTS_IN"] = info.FIELD_STARTS_IN
    config["FIELD_DIMS_IN"] = info.FIELD_DIMS_IN
    config["FIELD_STARTS_NET"] = info.FIELD_STARTS_NET
    config["FIELD_DIMS_NET"] = info.FIELD_DIMS_NET
    config["ACTIVATIONS"] = info.ACTIVATIONS

    lstm = Encoder_Decoder_lstm(config, n_feat_inp, conditional=True)
    train = Train(lstm)
    train.train(train_batches, val_batches, epochs=epochs, early_stop=early_stop)
    attributes = encoder.attributes

    # synth = train.generate_synthetic_data(len_generated_seq, num_generated_seq, df, attributes, n_feat_inp)
    # #synth = train.generate_synthetic_tcode(len_generated_seq,num_generated_seq, df, attributes, n_feat_inp)
    # #synth = train.generate_synthetic_data_type2(len_generated_seq,num_generated_seq, df, attributes, n_feat_inp)

    # synth.to_csv(synth_data_filename, index=False)
    
    # log_parameters(synth_data_filename, confighyper)
    # print('finish')
    


Finished encoding 2000 of 10995 seqs
Finished encoding 4000 of 10995 seqs
Finished encoding 6000 of 10995 seqs
Finished encoding 8000 of 10995 seqs
Finished encoding 10000 of 10995 seqs
Took 25.68 secs
Epoch 1 Batch0 Loss 14.3662
Epoch 1 Batch50 Loss 6.9773
Epoch 1 Batch100 Loss 6.5392
Epoch 1 Loss 6.3725
** on validation data loss is 5.8485
Time taken for 1 epoch: 5.88 secs

Epoch 2 Batch0 Loss 5.8371
Epoch 2 Batch50 Loss 5.7878
Epoch 2 Batch100 Loss 5.7411
Epoch 2 Loss 5.7061
** on validation data loss is 5.6033
Time taken for 1 epoch: 5.76 secs



In [9]:
lstm.summary()

Model: "encoder__decoder_lstm_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                multiple                  79360     
_________________________________________________________________
lstm_9 (LSTM)                multiple                  131584    
_________________________________________________________________
lstm_10 (LSTM)               multiple                  131584    
_________________________________________________________________
lstm_11 (LSTM)               multiple                  131584    
_________________________________________________________________
dense_17 (Dense)             multiple                  4619      
_________________________________________________________________
dense_15 (Dense)             multiple                  1015      
_________________________________________________________________
dense_18 (Dense)             multiple      

In [3]:
import pandas as pd
loss = pd.read_csv('loss_lstm_dp1.csv')
loss

,loss,val_loss
0,6.334760,5.957420
1,5.758039,5.629686
2,5.579495,5.510149
3,5.447209,5.385936
4,5.317694,5.249732
...,...,...
61,4.246257,4.388506
62,4.280646,4.337390
63,4.223671,4.380812
64,4.217332,4.316156


In [3]:
info = FieldInfo_type2()
info.FIELD_DIMS_IN, info.FIELD_DIMS_NET, info.FIELD_DIMS_TAR

({'tcode_num': 16, 'td_sc': 1, 'log_amount_sc': 1},
 {'tcode_num': 16, 'td_sc': 2, 'log_amount_sc': 2},
 {'tcode_num': 1, 'td_sc': 1, 'log_amount_sc': 1})

In [3]:
raw_data = pd.read_csv('../DATA/tr_by_acct_w_age.csv')
data, LOG_AMOUNT_SCALE, TD_SCALE,ATTR_SCALE, START_DATE, TCODE_TO_NUM, NUM_TO_TCODE = preprocess_data_czech(raw_data)
data2 = data[['account_id','age','age_sc', 'tcode', 'tcode_num', 'datetime', 'month', 'dow', 'day','td', 'dtme', 'log_amount','log_amount_sc','td_sc']]
df= data2.copy()

max_seq_len = 80
min_seq_len = 20

encoder = TensorEncoder(df, info, max_seq_len, min_seq_len)


In [4]:
encoder.encode()

Finished encoding 2000 of 14354 seqs
Finished encoding 4000 of 14354 seqs
Finished encoding 6000 of 14354 seqs
Finished encoding 8000 of 14354 seqs
Finished encoding 10000 of 14354 seqs
Finished encoding 12000 of 14354 seqs
Finished encoding 14000 of 14354 seqs
Took 38.53 secs


In [5]:
encoder.inp_tensor.shape, encoder.tar_tensor.shape

((14354, 81, 26), (14354, 80, 11))

In [6]:
def make_batches(ds, buffer_size, batch_size):
    return ds.cache().shuffle(buffer_size).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

n_seqs, n_steps, n_feat_inp = encoder.inp_tensor.shape
x_tr, x_cv, inds_tr, inds_cv, targ_tr, targ_cv = train_test_split(encoder.inp_tensor, np.arange(n_seqs), encoder.tar_tensor, test_size=0.2)

# Create TensorFlow dataset
ds_all = tf.data.Dataset.from_tensor_slices((encoder.inp_tensor.astype(np.float32), encoder.tar_tensor.astype(np.float32)))
ds_tr = tf.data.Dataset.from_tensor_slices((x_tr.astype(np.float32), targ_tr.astype(np.float32)))
ds_cv = tf.data.Dataset.from_tensor_slices((x_cv.astype(np.float32), targ_cv.astype(np.float32)))

BUFFER_SIZE = ds_all.cardinality().numpy()
bs = 64  # batch size


train_batches = make_batches(ds_tr, BUFFER_SIZE, bs)
val_batches =  make_batches(ds_cv, BUFFER_SIZE, bs)

In [7]:
i = 1
for (batch_no, (inp, tar)) in enumerate(train_batches):
    if i == 1:
        break

In [8]:
config = {}
config["ORDER"] = info.DATA_KEY_ORDER
config["FIELD_STARTS_IN"] = info.FIELD_STARTS_IN
config["FIELD_DIMS_IN"] = info.FIELD_DIMS_IN
config["FIELD_STARTS_NET"] = info.FIELD_STARTS_NET
config["FIELD_DIMS_NET"] = info.FIELD_DIMS_NET
config["ACTIVATIONS"] = info.ACTIVATIONS
inp_feat = sum(info.FIELD_DIMS_IN.values())
lstm = Encoder_Decoder_lstm(config, inp_feat, conditional=True)
train = Train(lstm)

In [10]:
preds = lstm(inp)
real = tar

In [11]:
# loss function
real = tar
INP_ENCODING = info.INP_ENCODINGS
NET_ENCODING = info.NET_ENCODINGS
FIELD_STARTS_TAR = info.FIELD_STARTS_TAR
FIELD_DIMS_TAR = info.FIELD_DIMS_TAR
LOSS_TYPES = info.LOSS_TYPES
FIELD_STARTS_IN = info.FIELD_STARTS_IN
FIELD_DIMS_IN = info.FIELD_DIMS_IN
FIELD_DIMS_NET = info.FIELD_DIMS_NET
loss_scce_logit = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
loss_mse = tf.keras.losses.MeanSquaredError(reduction='none')

def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return  -.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi)

LOSS_WEIGHTS = {
 'td_sc':1.,
 'month': 0.015,
 'day': 0.025,
 'dtme': 0.025,
 'dow': 0.01,
 'tcode_num': 1.,
 'log_amount_sc': 2.}

loss_parts = []
loss_parts_weighted = []
mask = tf.math.logical_not(tf.math.equal(tf.reduce_sum(real, axis=2), 0))
for k, k_pred in preds.items():
    st = FIELD_STARTS_TAR[k]
    end = st + FIELD_DIMS_TAR[k]
    loss_type = LOSS_TYPES[k]
    if loss_type == "scce":
        loss_ = loss_scce_logit(real[:, :, st:end], k_pred)
    elif loss_type == "pdf":
        loss_ = -log_normal_pdf(real[:, :, st:end], k_pred[:,:,0:1], k_pred[:,:,1:2])[:,:,0]
    elif loss_type == 'mse':
        loss_ = loss_mse(real[:, :, st:end], k_pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    loss_ = tf.reduce_sum(loss_)/tf.reduce_sum(mask) 

    loss_parts.append(loss_)
    loss_parts_weighted.append(loss_ * LOSS_WEIGHTS[k])
#return tf.reduce_sum(loss_parts_weighted)

In [12]:
train.train(train_batches, val_batches, epochs=5, early_stop=5)

2023-12-30 09:54:06.678997: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x24cf11e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-30 09:54:06.679022: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 8000, Compute Capability 7.5
2023-12-30 09:54:06.679026: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Quadro RTX 8000, Compute Capability 7.5
2023-12-30 09:54:06.683211: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-30 09:54:06.780018: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1 Batch0 Loss 21.7639
Epoch 1 Batch50 Loss 6.3287
Epoch 1 Batch100 Loss 5.6140
Epoch 1 Batch150 Loss 5.2996
Epoch 1 Loss 5.1793
** on validation data loss is 4.4675
Time taken for 1 epoch: 18.14 secs

Epoch 2 Batch0 Loss 4.4306
Epoch 2 Batch50 Loss 4.4150
Epoch 2 Batch100 Loss 4.3478
Epoch 2 Batch150 Loss 4.3036
Epoch 2 Loss 4.2862
** on validation data loss is 4.1555
Time taken for 1 epoch: 15.29 secs

Epoch 3 Batch0 Loss 4.2472
Epoch 3 Batch50 Loss 4.0839
Epoch 3 Batch100 Loss 4.0508
Epoch 3 Batch150 Loss 4.0077
Epoch 3 Loss 3.9831
** on validation data loss is 3.8243
Time taken for 1 epoch: 15.84 secs

Epoch 4 Batch0 Loss 3.8001
Epoch 4 Batch50 Loss 3.8267
Epoch 4 Batch100 Loss 3.8174
Epoch 4 Batch150 Loss 3.7922
Epoch 4 Loss 3.7887
** on validation data loss is 3.7135
Time taken for 1 epoch: 15.65 secs

Epoch 5 Batch0 Loss 3.7687
Epoch 5 Batch50 Loss 3.7265
Epoch 5 Batch100 Loss 3.6922
Epoch 5 Batch150 Loss 3.6631
Epoch 5 Loss 3.6584
** on validation data loss is 3.5993
Time 

In [13]:
attributes = encoder.attributes
attributes.shape

(14354,)

In [14]:
#generate synthetic data
n_seqs_to_generate = 3
attributes = encoder.attributes
max_length = 25

MAX_YEARS_SPAN = 15
get_dtme = lambda d: calendar.monthrange(d.year, d.month)[1] - d.day

START_DATE = df["datetime"].min()
ATTR_SCALE = df["age"].std()
LOG_AMOUNT_SCALE = df["log_amount"].std()
TD_SCALE = df["td"].std()
NUM_TO_TCODE = dict([(i, tc) for i, tc in enumerate(df['tcode'].unique())])

END_DATE = START_DATE.replace(year = START_DATE.year+ MAX_YEARS_SPAN)

ALL_DATES = [START_DATE + datetime.timedelta(i) for i in range((END_DATE - START_DATE).days)]
AD = np.array([(d.month % 12, d.day % 31, d.weekday() % 7, i, d.year, get_dtme(d)) for i, d in enumerate(ALL_DATES)])
start_date_opts = df.groupby("account_id")["datetime"].min().dt.date.to_list()   #len = 4500
start_dates = np.random.choice(start_date_opts, size=n_seqs_to_generate) # sample start dates from real data

seq_ages = np.random.choice(attributes, size=n_seqs_to_generate) # sample ages from real data

#generate sequences
start_inds = np.array([(d - START_DATE.date()).days for d in start_dates])    #array of shape (n_seqs_to_generate,)
#print(start_inds)
inp = np.repeat(np.array(seq_ages)[:, None, None], repeats=n_feat_inp, axis=2) / ATTR_SCALE   #(n_seqs_to_generate, 1, n_feat_inp) 
raw_date_info_list = []

conditional = train.lstm.conditional
inference_model = Encoder_Decoder_lstm_Inference(train.lstm.config, train.lstm.inp_feat, conditional)
dummy_input = tf.random.normal([n_seqs_to_generate, max_length, n_feat_inp])
# Build the model by running dummy data through it
inference_model(dummy_input)
inference_model.load_weights('lstm_model_weights.h5')


In [15]:
def bulk_encode_time_value(val, max_val):
    """ encoding date features in the clockwise dimension """
    x = np.sin(2 * np.pi / max_val * val)
    y = np.cos(2 * np.pi / max_val * val)
    return np.stack([x, y], axis=1)

def clock_to_probs(pt, pts):
    EPS_CLOCKP = 0.01
    pt_expanded = tf.expand_dims(pt, 2)                     # Shape: (n_seq, seq_len, 1, 2)
    pts_expanded = tf.reshape(pts, [1, 1, pts.shape[0], 2]) # Shape: (1, 1, net_dim, 2)

    ds = tf.constant(pts_expanded) - pt_expanded
    print(ds.shape)
    sq_ds = np.sum(tf.square(ds+EPS_CLOCKP), axis= -1)
    print(sq_ds.shape)
    raw_ps = 1/ sq_ds   
    return raw_ps / np.sum(raw_ps)

def reencode_net_prediction(net_name, predictions):
    """net_name is in ['tcode_num', 'dow', 'month', 'day', 'dtme', 'td_sc', 'log_amount_sc']
       predictions for ['tcode_num', 'dow', 'month', 'day', 'dtme'] are probabilities of dimensions [16, 7, 12, 31, 31]

       function:  transform predictions to the correct form to be used for conditional generating: 
                  convert 'tcode' to one-hot encoded vector, 
                  convert date features to clock dimension
                  extract the predicted mean of 'td' and 'amount' as predicted values 
                
    """
    date_info = {'month':12, 'day':31, 'dtme':31, 'dow':7}
    CLOCKS = {}
    for k, val in date_info.items():
        CLOCKS[k] = tf.constant(bulk_encode_time_value(np.arange(val), val), dtype=tf.float32)
        
    batch_size = predictions.shape[0]
    print('prediction shape', predictions.shape)
    if "_num" in net_name:
        dim = FIELD_DIMS_NET[net_name]
        choices = np.arange(dim)
        ps = tf.nn.softmax(predictions, axis=2).numpy().reshape(-1, dim)    #predictions: (n_seq_to_generate, seq_len, dim=16)
        choosen =  np.reshape([np.random.choice(choices, p=p) for p in ps], newshape=(batch_size, -1))

        return tf.one_hot(choosen, depth=dim)      #(n_seq_to_generate, seq_len, dim=16)
    
    elif net_name in date_info.keys() and INP_ENCODING[net_name] == 'cl' and 'oh' in NET_ENCODING[net_name] :

        dim = FIELD_DIMS_NET[net_name]
        choices = np.arange(dim)
        ps = tf.nn.softmax(predictions, axis=2).numpy().reshape(-1, dim)
        choosen =  np.array([np.random.choice(choices, p=p) for p in ps])
        x = bulk_encode_time_value(choosen, max_val=dim)
        
        return np.reshape(x, newshape=(batch_size, -1, 2))
    
    elif net_name in date_info.keys() and 'oh' in INP_ENCODING[net_name] and 'oh' in NET_ENCODING[net_name]:
        dim = FIELD_DIMS_NET[net_name]
        assert predictions.shape[-1] == dim
        choices = np.arange(dim)
        ps = tf.nn.softmax(predictions, axis=2).numpy().reshape(-1, dim)    #predictions: (n_seq_to_generate, seq_len, dim=dim)
        choosen =  np.reshape([np.random.choice(choices, p=p) for p in ps], newshape=(batch_size, -1))
        
        return tf.one_hot(choosen, depth=dim) 
    elif net_name in date_info.keys() and 'cl' in INP_ENCODING[net_name] and 'cl' in NET_ENCODING[net_name]:
        dim = date_info[net_name]
        assert predictions.shape[-1] == FIELD_DIMS_NET[net_name]
        choices = np.arange(dim)
        print(CLOCKS[net_name])
        print(predictions.shape)
        preds = clock_to_probs(predictions, CLOCKS[net_name])
        print(preds.shape)
        ps = tf.nn.softmax(preds, axis=2).numpy().reshape(-1, dim)    #predictions: (n_seq_to_generate, seq_len, dim=dim)
        choosen =  np.reshape([np.random.choice(choices, p=p) for p in ps], newshape=(batch_size, -1))
        x = bulk_encode_time_value(choosen, max_val=dim)
        
        return np.reshape(x, newshape=(batch_size, -1, 2))
    elif net_name in ['td_sc', "log_amount_sc"]:
        return predictions[:, :, 0:1]


In [16]:
lstm = inference_model
final_output = lstm(inp, return_decoder_lstm2_output=True)

### Predict each field  ###

#raw_preds is the outputs of the last dense layer of deep model. 
raw_preds = {}
#preds is the reencoded raw_preds, 'tcode' converts to one-hot encoded, 'date-features' are converted to clock-wise
#and for 'amount' and 'td' the predicted mean is extracted. it is used for conditional generating. 
preds = {}
#encoded_preds_d is similar to preds for 'tcode', 'td', and 'amount', but for date features , the predicted date is computed 
#based on a formula 
encoded_preds_d = {}

if lstm.conditional:
    for net_name in lstm.ORDER:  

        raw_preds[net_name] = lstm.dense_layers[net_name](final_output)
        en_pred = reencode_net_prediction(net_name, raw_preds[net_name]) 
        preds[net_name] = en_pred
            
        encoded_preds_d[net_name] = en_pred[:,-1,:] 
        #encoded_preds.append(pred[:,-1,:])
        final_output = tf.concat([final_output, en_pred], axis=2)

prediction shape (3, 1, 16)
prediction shape (3, 1, 2)
tf.Tensor(
[[ 0.          1.        ]
 [ 0.7818315   0.6234898 ]
 [ 0.9749279  -0.22252093]
 [ 0.43388373 -0.90096885]
 [-0.43388373 -0.90096885]
 [-0.9749279  -0.22252093]
 [-0.7818315   0.6234898 ]], shape=(7, 2), dtype=float32)
(3, 1, 2)
(3, 1, 7, 2)
(3, 1, 7)
(3, 1, 7)
prediction shape (3, 1, 2)
tf.Tensor(
[[ 0.0000000e+00  1.0000000e+00]
 [ 5.0000000e-01  8.6602539e-01]
 [ 8.6602539e-01  5.0000000e-01]
 [ 1.0000000e+00  6.1232343e-17]
 [ 8.6602539e-01 -5.0000000e-01]
 [ 5.0000000e-01 -8.6602539e-01]
 [ 1.2246469e-16 -1.0000000e+00]
 [-5.0000000e-01 -8.6602539e-01]
 [-8.6602539e-01 -5.0000000e-01]
 [-1.0000000e+00 -1.8369701e-16]
 [-8.6602539e-01  5.0000000e-01]
 [-5.0000000e-01  8.6602539e-01]], shape=(12, 2), dtype=float32)
(3, 1, 2)
(3, 1, 12, 2)
(3, 1, 12)
(3, 1, 12)
prediction shape (3, 1, 2)
tf.Tensor(
[[ 0.          1.        ]
 [ 0.20129852  0.9795299 ]
 [ 0.39435586  0.9189578 ]
 [ 0.5712682   0.82076347]
 [ 0.7247928 

In [39]:
lstm.conditional

False

In [37]:
#call to generate 
lstm = inference_model
final_output = lstm(inp, return_decoder_lstm2_output=True)

### Predict each field  ###

#raw_preds is the outputs of the last dense layer of deep model. 
raw_preds = {}
#preds is the reencoded raw_preds, 'tcode' converts to one-hot encoded, 'date-features' are converted to clock-wise
#and for 'amount' and 'td' the predicted mean is extracted. it is used for conditional generating. 
preds = {}
#encoded_preds_d is similar to preds for 'tcode', 'td', and 'amount', but for date features , the predicted date is computed 
#based on a formula 
encoded_preds_d = {}

if lstm.conditional:
    for net_name in lstm.ORDER:  

        raw_preds[net_name] = lstm.dense_layers[net_name](final_output)
        en_pred = reencode_net_prediction(net_name, raw_preds[net_name]) 
        preds[net_name] = en_pred
            
        encoded_preds_d[net_name] = en_pred[:,-1,:] 
        #encoded_preds.append(pred[:,-1,:])
        final_output = tf.concat([final_output, en_pred], axis=2)

    
else:
    final_output = lstm.dense_layer(final_output)
    st = 0
    for net_name, dim in lstm.FIELD_DIMS_NET.items():
        acti = lstm.ACTIVATIONS.get(net_name, None)
        ed = st + dim
        if acti is None:
            raw_preds[net_name] = final_output[:,:,st:ed]
        elif acti == 'relu':
            raw_preds[net_name] = tf.keras.activations.relu(final_output[:, :, st:ed])
        st = ed

        en_pred = reencode_net_prediction(net_name, raw_preds[net_name]) 
        preds[net_name] = en_pred

        encoded_preds_d[net_name] = en_pred[:,-1,:] 

In [36]:
inp.shape

(3, 1, 26)

In [3]:

def bulk_encode_time_value(val, max_val):
    """ encoding date features in the clockwise dimension """
    x = np.sin(2 * np.pi / max_val * val)
    y = np.cos(2 * np.pi / max_val * val)
    return np.stack([x, y], axis=1)
date_info = {'month':12, 'day':31, 'dtme':31, 'dow':7}
CLOCKS = {}
for k, val in date_info.items():
    CLOCKS[k] = tf.constant(bulk_encode_time_value(np.arange(val), val), dtype=tf.float32)

2023-12-29 09:59:32.560111: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 47372 MB memory:  -> device: 0, name: Quadro RTX 8000, pci bus id: 0000:3b:00.0, compute capability: 7.5
2023-12-29 09:59:32.560603: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 47372 MB memory:  -> device: 1, name: Quadro RTX 8000, pci bus id: 0000:af:00.0, compute capability: 7.5


In [ ]:
def clock_to_probs(pt, pts):
    EPS_CLOCKP = 0.01
    ds = tf.constant(pts) - pt
    sq_ds = np.sum(tf.square(ds+EPS_CLOCKP), axis=1)
    raw_ps = 1/ sq_ds   
    return raw_ps / np.sum(raw_ps)


In [6]:
pts = CLOCKS['month']


In [8]:
epochs = 5
early_stop = 3
train.train(train_batches, val_batches, epochs, early_stop)

2023-12-27 23:04:08.029628: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-12-27 23:04:08.304819: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x23cf2990 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-27 23:04:08.304843: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 8000, Compute Capability 7.5
2023-12-27 23:04:08.304846: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): Quadro RTX 8000, Compute Capability 7.5
2023-12-27 23:04:08.329707: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-27 23:04:08.544273: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1 Batch0 Loss 14.9649
Epoch 1 Batch50 Loss 7.2448
Epoch 1 Batch100 Loss 6.8731
Epoch 1 Batch150 Loss 6.7448


ValueError: Exception encountered when calling layer 'encoder__decoder_lstm' (type Encoder_Decoder_lstm).

Input 0 of layer "lstm" is incompatible with the layer: expected shape=(64, None, 26), found shape=(27, 80, 26)

Call arguments received by layer 'encoder__decoder_lstm' (type Encoder_Decoder_lstm):
  • inp=tf.Tensor(shape=(27, 81, 26), dtype=float32)

In [8]:
lstm.summary()

NameError: name 'lstm' is not defined

In [ ]:
max_length = 25
n_seqs_to_generate = 3
attributes = encoder.attributes
n_seqs, seq_len, n_feat_inp = encoder.inp_tensor.shape
train.generate_synthetic_data(max_length, n_seqs_to_generate, df, attributes, n_feat_inp)

## evaluate

In [1]:
import sys
sys.path.insert(0, '/users/fs2/hmehri/pythonproject/Thesis/synthetic')

import pandas as pd
import matplotlib.pyplot as plt
from lib.prepare_data import preprocess_data_czech
from lib.eval import comapre_unidist_cont, compute_ngram_metrics, comapre_unidist_cat, compute_2d_categorical_metrics

In [2]:
raw_data = pd.read_csv('../DATA/tr_by_acct_w_age.csv')
raw_data = raw_data.sort_values(by = ["account_id", "date"])
data, LOG_AMOUNT_SCALE, TD_SCALE,ATTR_SCALE, START_DATE, _ = preprocess_data_czech(raw_data)
real = data[['account_id','tcode', 'datetime','year', 'month', 'dow', 'day','td', 'dtme', 'amount', 'raw_amount']]
real_cf = real[["account_id", "month", "raw_amount", "year"]].groupby(["account_id", "month", "year"],as_index=False)["raw_amount"].sum()
real_sorted = real.sort_values(['account_id', 'year', 'month', 'day'])
real

,account_id,tcode,datetime,year,month,dow,day,td,dtme,amount,raw_amount
0,1,CREDIT__CREDIT IN CASH__nan,1995-03-24,1995,3,4,24,0.0,7,1000.0,1000.0
1,1,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1995-04-13,1995,4,3,13,20.0,17,3679.0,3679.0
2,1,CREDIT__CREDIT IN CASH__nan,1995-04-23,1995,4,6,23,10.0,7,12600.0,12600.0
3,1,CREDIT__nan__INTEREST CREDITED,1995-04-30,1995,4,6,30,7.0,0,19.2,19.2
4,1,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1995-05-13,1995,5,5,13,13.0,18,3679.0,3679.0
...,...,...,...,...,...,...,...,...,...,...,...
1056315,11382,DEBIT__CASH WITHDRAWAL__nan,1998-12-02,1998,0,2,2,2.0,29,25600.0,-25600.0
1056316,11382,CREDIT__COLLECTION FROM ANOTHER BANK__nan,1998-12-10,1998,0,3,10,8.0,21,46248.0,46248.0
1056317,11382,DEBIT__CASH WITHDRAWAL__nan,1998-12-25,1998,0,4,25,15.0,6,6300.0,-6300.0
1056318,11382,CREDIT__nan__INTEREST CREDITED,1998-12-31,1998,0,3,0,6.0,0,311.3,311.3


In [3]:
synth = pd.read_csv('synth_lstm_dp1.csv')
synth

,amount,transaction_code,account_id,year,month,day,date,days_passed
0,0.37,CREDIT__nan__INTEREST CREDITED,0,1993,3,28,1993-03-28,0
1,2.75,DEBIT__CASH WITHDRAWAL__,0,1993,3,29,1993-03-29,1
2,78.12,CREDIT__COLLECTION FROM ANOTHER BANK__OLD AGE ...,0,1993,4,5,1993-04-05,7
3,17.20,CREDIT__nan__INTEREST CREDITED,0,1993,4,30,1993-04-30,25
4,3551.89,DEBIT__CASH WITHDRAWAL__nan,0,1993,5,11,1993-05-11,11
...,...,...,...,...,...,...,...,...
399995,62.29,CREDIT__nan__INTEREST CREDITED,4999,1998,9,30,1998-09-30,30
399996,4648.60,CREDIT__CREDIT IN CASH__nan,4999,1998,10,16,1998-10-16,16
399997,2223.25,DEBIT__CASH WITHDRAWAL__nan,4999,1998,11,4,1998-11-04,19
399998,61.84,CREDIT__nan__INTEREST CREDITED,4999,1998,11,30,1998-11-30,26


In [4]:
# synth = pd.read_csv('synth_lstm_tcode_v1.csv')
synth.rename(columns={'days_passed': 'td', 'transaction_code': 'tcode'}, inplace=True)
synth['type'] = synth['tcode'].str.split('__').str[0]
synth['raw_amount'] = synth.apply(lambda row: row['amount'] if row['type'] == 'CREDIT' else -row['amount'], axis=1)

synth_sorted = synth.sort_values(['account_id', 'year', 'month', 'day'])

synth_cf = synth[["account_id", "month", "raw_amount", "year"]].groupby(["account_id", "month", "year"],as_index=False)["raw_amount"].sum()

In [2]:
import pandas as pd
synth = pd.read_csv('synth_lstm_tcode.csv')
synth

,tcode,account_id
0,DEBIT__CASH WITHDRAWAL__INSURANCE PAYMENT,0
1,DEBIT__CREDIT CARD WITHDRAWAL__nan,0
2,DEBIT__CASH WITHDRAWAL__nan,0
3,CREDIT__nan__INTEREST CREDITED,0
4,DEBIT__REMITTANCE TO ANOTHER BANK__LOAN PAYMENT,0
...,...,...
399995,DEBIT__REMITTANCE TO ANOTHER BANK__,4999
399996,DEBIT__REMITTANCE TO ANOTHER BANK__,4999
399997,DEBIT__CASH WITHDRAWAL__nan,4999
399998,DEBIT__REMITTANCE TO ANOTHER BANK__HOUSEHOLD,4999


In [5]:
synth = pd.read_csv('synth_lstm_v3.csv')
cat_code_fields = ['type', 'operation', 'k_symbol']
TCODE_SEP = "__"
# create tcode by concating fields in "cat_code_fields"
tcode = synth[cat_code_fields[0]].astype(str)
for ccf in cat_code_fields[1:]:
    tcode += TCODE_SEP + synth[ccf].astype(str)

synth["tcode"] = tcode

synth.rename(columns={'days_passed': 'td'}, inplace=True)
synth['raw_amount'] = synth.apply(lambda row: row['amount'] if row['type'] == 'CREDIT' else -row['amount'], axis=1)

synth_sorted = synth.sort_values(['account_id', 'year', 'month', 'day'])

synth_cf = synth[["account_id", "month", "raw_amount", "year"]].groupby(["account_id", "month", "year"],as_index=False)["raw_amount"].sum()
synth

,amount,k_symbol,operation,type,account_id,year,month,day,date,td,tcode,raw_amount
0,7.38,HOUSEHOLD,CREDIT CARD WITHDRAWAL,DEBIT,0,1996,9,29,1996-09-29,0,DEBIT__CREDIT CARD WITHDRAWAL__HOUSEHOLD,-7.38
1,92.91,_nan,CASH WITHDRAWAL,DEBIT,0,1996,10,2,1996-10-02,3,DEBIT__CASH WITHDRAWAL___nan,-92.91
2,0.13,PAYMENT ON STATEMENT,CASH WITHDRAWAL,DEBIT,0,1996,10,31,1996-10-31,29,DEBIT__CASH WITHDRAWAL__PAYMENT ON STATEMENT,-0.13
3,1543.77,_nan,CREDIT CARD WITHDRAWAL,DEBIT,0,1996,11,7,1996-11-07,7,DEBIT__CREDIT CARD WITHDRAWAL___nan,-1543.77
4,213.19,INTEREST CREDITED,_nan,CREDIT,0,1996,11,9,1996-11-09,2,CREDIT___nan__INTEREST CREDITED,213.19
...,...,...,...,...,...,...,...,...,...,...,...,...
399995,861.83,_nan,COLLECTION FROM ANOTHER BANK,CREDIT,4999,1995,6,14,1995-06-14,0,CREDIT__COLLECTION FROM ANOTHER BANK___nan,861.83
399996,290.86,_nan,CASH WITHDRAWAL,DEBIT,4999,1995,6,17,1995-06-17,3,DEBIT__CASH WITHDRAWAL___nan,-290.86
399997,279.76,_nan,REMITTANCE TO ANOTHER BANK,DEBIT,4999,1995,6,18,1995-06-18,1,DEBIT__REMITTANCE TO ANOTHER BANK___nan,-279.76
399998,1062.36,_nan,COLLECTION FROM ANOTHER BANK,CREDIT,4999,1995,6,22,1995-06-22,4,CREDIT__COLLECTION FROM ANOTHER BANK___nan,1062.36


In [7]:
synth = pd.read_csv('synth_lstm_dp1.csv')
synth

,amount,transaction_code,account_id,year,month,day,date,days_passed
0,0.37,CREDIT__nan__INTEREST CREDITED,0,1993,3,28,1993-03-28,0
1,2.75,DEBIT__CASH WITHDRAWAL__,0,1993,3,29,1993-03-29,1
2,78.12,CREDIT__COLLECTION FROM ANOTHER BANK__OLD AGE ...,0,1993,4,5,1993-04-05,7
3,17.20,CREDIT__nan__INTEREST CREDITED,0,1993,4,30,1993-04-30,25
4,3551.89,DEBIT__CASH WITHDRAWAL__nan,0,1993,5,11,1993-05-11,11
...,...,...,...,...,...,...,...,...
399995,62.29,CREDIT__nan__INTEREST CREDITED,4999,1998,9,30,1998-09-30,30
399996,4648.60,CREDIT__CREDIT IN CASH__nan,4999,1998,10,16,1998-10-16,16
399997,2223.25,DEBIT__CASH WITHDRAWAL__nan,4999,1998,11,4,1998-11-04,19
399998,61.84,CREDIT__nan__INTEREST CREDITED,4999,1998,11,30,1998-11-30,26


In [9]:
from datetime import datetime

czech_date_parser = lambda x: datetime.strptime(str(x), "%Y-%m-%d")
synth["datetime"] = synth["date"].apply(czech_date_parser)
synth["month"] = synth["datetime"].dt.month 
synth["day"] = synth["datetime"].dt.day 
synth["dow"] =  synth["datetime"].dt.dayofweek 
synth["year"] = synth["datetime"].dt.year
synth['tcode'] = synth['transaction_code']
synth.rename(columns={'days_passed': 'td'}, inplace=True)
synth['type'] = synth['tcode'].str.split('__').str[0]
synth['raw_amount'] = synth.apply(lambda row: row['amount'] if row['type'] == 'CREDIT' else -row['amount'], axis=1)

synth_sorted = synth.sort_values(['account_id', 'year', 'month', 'day'])

synth_cf = synth[["account_id", "month", "raw_amount", "year"]].groupby(["account_id", "month", "year"],as_index=False)["raw_amount"].sum()
    

In [ ]:
synth[synth['account_id'] == 0]

In [10]:
CONT_FIELDS = ["amount", "td"]

CF_FIELD = 'raw_amount'

#compare univariate distribution of continuous columns
comapre_unidist_cont(CONT_FIELDS,CF_FIELD, real, synth, real_cf, synth_cf)

{'amount': {'wasser': 4174.573380036866,
  'ks': 0.24605198424719776,
  'energy_d': 34.9193550921508},
 'td': {'wasser': 10.252238490608907,
  'ks': 0.25543474250227205,
  'energy_d': 1.8358876046889725},
 'CF': {'wasser': 4226.028779972578,
  'ks': 0.15893154299225914,
  'energy_d': 24.163848319263757}}

In [11]:
# JSD between the distributions of tcode 3-grams
synth_sorted = synth
combo_df, result = compute_ngram_metrics(real_sorted, synth_sorted, 'tcode', 3)
result

{'jsd': 0.24065076970017257,
 'entr_r': 5.351895164576639,
 'entr_g': 5.4837167582921555,
 'NED': -0.13182159371551627,
 'l1': 1.1322630832101863,
 'l2': 0.11318215558365517,
 'jac': 0.5880572372041827,
 'count_r': 1548,
 'coverage_r': 0.3779296875,
 'count_g': 3583,
 'coverage_g': 0.874755859375,
 'count_max': 4096,
 'field': 'tcode',
 'n': 3,
 'pseudo_counts': 0.0}

In [14]:
# JSD result comparing the univariate distributions of the tcode (Tcode), and DOM
#CAT_FIELDS = ['tcode', 'day', 'month']
CAT_FIELDS = ['tcode', 'day']
result_jst_cat = {}
for field in CAT_FIELDS:
    result_jst_cat[field] = comapre_unidist_cat(real, synth, field)
result_jst_cat

{'tcode': 0.0353668998874559, 'day': 0.14706534597241305}

In [13]:
#joint distribution of two categorical columns(tcode, DOM)
field1 = 'tcode'
field2 = 'day'
compute_2d_categorical_metrics(real, synth, field1, field2)

{'jsd': 0.2127952776525589,
 'entr_r': 4.3143515019943885,
 'entr_g': 4.663408977516541,
 'l1': 0.883125640714935,
 'l2': 0.19175228938017574,
 'jac': 0.5396825396825397,
 'count_r': 240.0,
 'coverage_r': 0.46875,
 'count_g': 496.0,
 'coverage_g': 0.96875,
 'count_max': 512}